## Optimization

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn as skl
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Determine the number of unique values in each column.
# create for loop using columns to create list
col_list = application_df.columns

for col in col_list:
  print(f"{col} Distribution")
  print(f"{application_df[col].value_counts()}\n")

EIN Distribution
10520599     1
626274659    1
630475330    1
630416100    1
630357662    1
            ..
383880377    1
383876652    1
383874900    1
383871370    1
996086871    1
Name: EIN, Length: 34299, dtype: int64

NAME Distribution
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGAT

In [3]:
## additional columns that can be dropped in addition to EIN and NAME given low impact on performance
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'. - original preprocessing
# secondary preprocessing...remove special considerations and 
application_df.drop(['EIN', 'STATUS', 'SPECIAL_CONSIDERATIONS'], axis=1, inplace=True)
# check that columns removed properly
application_df.columns

Index(['NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [4]:
# list of values to replace 1 to 24999
income_amts_to_replace_1to25k = ['1-9999', '10000-24999']

# list of values to replace >= 500000
income_amts_to_replace_ge500000 = ['1M-5M', '10M-50M', '5M-10M', '50M+']

# Replace in dataframe
for app in income_amts_to_replace_1to25k:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(app,"1 to 24999")

    # Replace in dataframe
for app in income_amts_to_replace_ge500000:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(app,">= 500000")

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
>= 500000         1519
1 to 24999        1271
Name: INCOME_AMT, dtype: int64

In [5]:
# Choose a cutoff value and create a list of names to be replaced

s = application_df['NAME'].value_counts()
application_df['NAME'] = np.where(application_df['NAME'].isin(s.index[s < 26]), "Other", application_df['NAME'])

print(f"Distribution of Names: {application_df['NAME'].value_counts()}")
print(f"Number of Unique Vales: {application_df['NAME'].nunique()}")

Distribution of Names: Other                                                                   23031
PARENT BOOSTER USA INC                                                   1260
TOPS CLUB INC                                                             765
UNITED STATES BOWLING CONGRESS INC                                        700
WASHINGTON STATE UNIVERSITY                                               492
                                                                        ...  
VETERANS OF FOREIGN WARS OF THE UNITED STATES AUXILIARY DEPARTMENT         28
OPTIMIST INTERNATIONAL                                                     27
AMERICAN LEGION AUXILIARY                                                  26
CALIFORNIA TEACHERS ASSOCIATION                                            26
FUTURE FARMERS OF AMERICA AND ITS STATE ASSOCIATIONS & LOCAL CHAPTER       26
Name: NAME, Length: 94, dtype: int64
Number of Unique Vales: 94


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
s = application_df['CLASSIFICATION'].value_counts()
application_df['CLASSIFICATION'] = np.where(application_df['CLASSIFICATION'].isin(s.index[s < 1000]), "Other", application_df['CLASSIFICATION'])

application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Other,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,Other,T3,Independent,C2000,Preservation,Co-operative,1 to 24999,108590,1
2,Other,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,Other,T3,CompanySponsored,C2000,Preservation,Trust,1 to 24999,6692,1
4,Other,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_dummies.IS_SUCCESSFUL.values
X = application_dummies.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
feature_number = len(X_train[0])
layerNodes1 = 75
layerNodes2 = 50
layerNodes3 = 25
layerNodes4 = 10
layerNodes5 = 5
nn_model = tf.keras.models.Sequential()



# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=layerNodes1, activation="relu", input_dim=feature_number))



# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=layerNodes2, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=layerNodes3, activation="relu"))

# Fourth hidden layer
nn_model.add(tf.keras.layers.Dense(units=layerNodes4, activation="sigmoid"))

# Fifth hidden layer
nn_model.add(tf.keras.layers.Dense(units=layerNodes5, activation="tanh"))


# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 75)                9825      
                                                                 
 dense_20 (Dense)            (None, 50)                3800      
                                                                 
 dense_21 (Dense)            (None, 25)                1275      
                                                                 
 dense_22 (Dense)            (None, 10)                260       
                                                                 
 dense_23 (Dense)            (None, 5)                 55        
                                                                 
 dense_24 (Dense)            (None, 1)                 6         
                                                                 
Total params: 15,221
Trainable params: 15,221
Non-trai

In [23]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
804/804 [==============================] - 4s 5ms/step - loss: 0.4445 - accuracy: 0.7808
Epoch 2/25
804/804 [==============================] - 2s 3ms/step - loss: 0.4439 - accuracy: 0.7816
Epoch 3/25
804/804 [==============================] - 2s 3ms/step - loss: 0.4433 - accuracy: 0.7815
Epoch 4/25
804/804 [==============================] - 2s 3ms/step - loss: 0.4438 - accuracy: 0.7822
Epoch 5/25
804/804 [==============================] - 2s 3ms/step - loss: 0.4435 - accuracy: 0.7824
Epoch 6/25
804/804 [==============================] - 2s 3ms/step - loss: 0.4441 - accuracy: 0.7816
Epoch 7/25
804/804 [==============================] - 2s 3ms/step - loss: 0.4442 - accuracy: 0.7814
Epoch 8/25
804/804 [==============================] - 2s 3ms/step - loss: 0.4440 - accuracy: 0.7816
Epoch 9/25
804/804 [==============================] - 2s 3ms/step - loss: 0.4437 - accuracy: 0.7823
Epoch 10/25
804/804 [==============================] - 2s 3ms/step - loss: 0.4434 - accuracy: 0.7826

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4854 - accuracy: 0.7666 - 349ms/epoch - 1ms/step
Loss: 0.4854350984096527, Accuracy: 0.766647219657898


In [32]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharityOptimized.h5")